In [47]:
import sys
import os
import torch
import pyspark 
import copy
import math
import rawpy
import random
import matplotlib
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from torch import nn
from PIL import Image
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2DTranspose, UpSampling2D, Conv2D
from tensorflow.keras.layers import LSTM,Embedding,Dropout,Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.python.keras import backend as K
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import lib
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input

torch.manual_seed(11)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
matplotlib.use('pdf')

# ./Training  2_class_fire_or_no_fire_winter

### https://ieee-dataport.org/open-access/flame-dataset-aerial-imagery-pile-burn-detection-using-drones-uavs

In [48]:

train_path = "./Training"
train_path_fire = train_path+ "/Fire"
train_path_no_fire = train_path+ "/No_Fire"


dataset = "./Training"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'mobilenet_v2'
hidden_layers = [128, 64, 32]
num_classes = 2
batch_size = 16
epochs = 10



In [49]:

def create_pdf(history, model_name):
    print("Generating pdf for " + model_name)

    # grab the desired metrics from tf history
    desired_metrics = ['accuracy', 'loss', 'val_accuracy', 'val_loss']
    metrics_to_plot = dict((k, history.history[k]) for k in desired_metrics if k in history.history)

    df = pd.DataFrame(metrics_to_plot)
    df.index = range(1, len(df)+1)
    df.rename(columns={'accuracy': 'Training Accuracy', 'loss': 'Training Loss',
        'val_accuracy': 'Validation Accuracy', 'val_loss': 'Validation Loss'}, inplace=True)
    print(df)
    sns.set()
    ax = sns.lineplot(marker='o', dashes=False, data=df)
    ax.set_xticks([1, 2, 3, 4, 5])
    ax.set_xlabel('Epoch')
    ax.set_yticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
    ax.set_ylabel('Value')
    ax.set_title('Model Loss and Accuracy')
    plt.savefig(f'./model_statistics/{model_name}_plot.pdf')

    print("Done")

In [57]:
METRICS = ['accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')]

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

baseModel = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )

model = Sequential()

# Transfer Learning
model.add(baseModel)
model.layers[0].trainable = False  
model.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2DTranspose(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))   
model.add(Conv2DTranspose(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2DTranspose(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))

model.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

model.add(Flatten())

for layer in range(len(hidden_layers)):
    model.add(Dense(hidden_layers[layer], activation='relu'))

# Add our classification layer and display model properties
model.add(Dense(num_classes, activation='softmax'))
model.summary()

#opt = tf.keras.optimizers.Adam(learning_rate=0.1)
# Compile the sections into one NN
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=METRICS)




Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_14 (Functional)       (None, 7, 7, 1280)        2257984   
                                                                 
 conv2d_transpose_54 (Conv2D  (None, 9, 9, 16)         184336    
 Transpose)                                                      
                                                                 
 conv2d_transpose_55 (Conv2D  (None, 11, 11, 32)       4640      
 Transpose)                                                      
                                                                 
 up_sampling2d_25 (UpSamplin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_transpose_56 (Conv2D  (None, 24, 24, 32)       9248      
 Transpose)                                          

In [58]:
# train the model
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

train_generator = data_generator.flow_from_directory(
    f'{dataset}',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')
num_training_files = len(train_generator.filepaths)


es_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
lrr= ReduceLROnPlateau(monitor='accuracy',   factor=.1,   patience=2,  min_lr=1e-5) 

# The division by 5 here is to make training quicker
# when you want maximum accuracy remove it.
history =  model.fit(train_generator, epochs=epochs, callbacks=[lrr,es_callback])

create_pdf(history, "mobilenet_v2")

Found 39375 images belonging to 2 classes.
Epoch 1/10
2461/2461 [==============================] - 169s 67ms/step - loss: 0.0675 - accuracy: 0.9755 - precision: 0.9755 - recall: 0.9755 - lr: 0.0100
Epoch 2/10
2461/2461 [==============================] - 166s 67ms/step - loss: 0.0136 - accuracy: 0.9954 - precision: 0.9954 - recall: 0.9954 - lr: 0.0100
Epoch 3/10
2461/2461 [==============================] - 171s 70ms/step - loss: 0.0057 - accuracy: 0.9981 - precision: 0.9981 - recall: 0.9981 - lr: 0.0100
Epoch 4/10
2461/2461 [==============================] - 168s 68ms/step - loss: 0.0046 - accuracy: 0.9983 - precision: 0.9983 - recall: 0.9983 - lr: 0.0100
Epoch 5/10
2461/2461 [==============================] - 163s 66ms/step - loss: 0.0017 - accuracy: 0.9994 - precision: 0.9994 - recall: 0.9994 - lr: 0.0100
Epoch 6/10
2461/2461 [==============================] - 169s 69ms/step - loss: 0.0010 - accuracy: 0.9997 - precision: 0.9997 - recall: 0.9997 - lr: 0.0100
Epoch 7/10
2461/2461 [=====

In [ ]:
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

print("Test" + model_name)
test_generator = data_generator.flow_from_directory(
                f'./Test',
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=False)
num_files = len(test_generator.filepaths)

steps = num_files/batch_size

model.evaluate(test_generator, steps=steps)


Testmobilenet_v2
Found 8617 images belonging to 2 classes.
408/538 [=====================>........] - ETA: 14s - loss: 2.3062 - accuracy: 0.7217 - precision: 0.7217 - recall: 0.7217

In [ ]:
model.save(f'./3_models_series/mobilenet_v2/{model_name}_2_fire_or_no_fire_winter.h5')


# ./FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Train 3_classes_fire_neutral_smoke

### https://github.com/DeepQuestAI/Fire-Smoke-Dataset

In [ ]:

dataset = "./FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Train"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'mobilenet_v2'
hidden_layers = [128, 64, 32]
num_classes = 3
batch_size = 64
epochs = 100


In [ ]:
METRICS = ['accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')]

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

baseModel = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )

model = Sequential()

# Transfer Learning
model.add(baseModel)
model.layers[0].trainable = True  
model.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2DTranspose(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))   
model.add(Conv2DTranspose(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

model.add(Flatten())

for layer in range(len(hidden_layers)):
    model.add(Dense(hidden_layers[layer], activation='relu'))

# Add our classification layer and display model properties
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compile the sections into one NN
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=METRICS)




In [ ]:
# train the model
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

train_generator = data_generator.flow_from_directory(
    f'{dataset}',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')
num_training_files = len(train_generator.filepaths)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
lrr= ReduceLROnPlateau(monitor='accuracy',   factor=.1,   patience=2,  min_lr=1e-5) 

# The division by 5 here is to make training quicker
# when you want maximum accuracy remove it.
history =  model.fit(train_generator, epochs=epochs, callbacks=[lrr,es_callback])



create_pdf(history, "mobilenet_v2")

In [ ]:
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

print("Test" + model_name)
test_generator = data_generator.flow_from_directory(
        f'FIRE-SMOKE-DATASET/FIRE-SMOKE-DATASET/Test',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
num_files = len(test_generator.filepaths)

steps = num_files/batch_size

model.evaluate(test_generator, steps=steps)

generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)

In [ ]:
model.save(f'./3_models_series/mobilenet_v2/{model_name}_3_fire_smoke_neutral.h5')


# E:\Facultate\Siva_master\Projects\git\change_detection\fire_smoke\Dataset\Training and Validation  2_class_fire_or_no_fire

### https://data.mendeley.com/datasets/gjmr63rz2r/1

In [ ]:
dataset = "./fire_smoke/Dataset/Training and Validation"
output_pdf = True
output_statistics = False
image_size = 224
model_name = 'mobilenet_v2'
hidden_layers = [128, 64, 32]
num_classes = 2
batch_size = 64
epochs = 1


In [ ]:
METRICS = ['accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')]

# baseModel = EfficientNetB7(
#     include_top=False, pooling='avg', weights='imagenet')

baseModel = tf.keras.applications.MobileNetV2(
    alpha=1.0,
    input_shape= (224, 224, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

baseModel = tf.keras.Model(baseModel.input, baseModel.get_layer(name = "out_relu").output )

model = Sequential()

# Transfer Learning
model.add(baseModel)
model.layers[0].trainable = True  
model.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2DTranspose(64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(UpSampling2D((2, 2)))   
model.add(Conv2DTranspose(128, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

model.add(Flatten())

for layer in range(len(hidden_layers)):
    model.add(Dense(hidden_layers[layer], activation='relu'))

# Add our classification layer and display model properties
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compile the sections into one NN
model.compile(optimizer='sgd', loss='categorical_crossentropy',
              metrics=METRICS)


In [ ]:
# train the model
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

train_generator = data_generator.flow_from_directory(
    f'{dataset}',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')
num_training_files = len(train_generator.filepaths)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)
lrr= ReduceLROnPlateau(monitor='accuracy',   factor=.1,   patience=2,  min_lr=1e-5) 

# The division by 5 here is to make training quicker
# when you want maximum accuracy remove it.
history =  model.fit(train_generator, epochs=epochs, callbacks=[lrr,es_callback])



create_pdf(history, "mobilenet_v2")

In [ ]:
data_generator = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

print("Test" + model_name)
test_generator = data_generator.flow_from_directory(
        f'fire_smoke/Dataset/Testing',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
num_files = len(test_generator.filepaths)

steps = num_files/batch_size

model.evaluate(test_generator, steps=steps)

generateStatistics(model, test_generator, model_name, num_classes, steps, loss, accuracy)



In [ ]:
model.save(f'./3_models_series/mobilenet_v2/{model_name}_2_fire_or_no_fire.h5')
